In [1]:
# Imports
# Standard Libraries of Python
import sys
import time
from collections import Counter

# Dependencies
import pandas as pd
import numpy as np

# Libraries made for this Proyect
from src.parse import draw_generator,Criteria

In [2]:
# Classes to be defined
class Tickets():
    def __init__(self,euromillions):
        self.recommended_numbers = euromillions.recommended_numbers
        self.not_recommended_numbers = euromillions.not_recommended_numbers

In [3]:
# Test of Tomorrow Numbers
euromillions = Criteria()

euromillions.groups_info()
euromillions.apply_transformation()
euromillions.count_skips()
euromillions.skips_for_last_12_draws()
euromillions.get_natural_rotations()
euromillions.numbers_clasification()

euromillions.year_criterion()
euromillions.rotation_criterion()
euromillions.position_criterion()
euromillions.group_criterion()
euromillions.numbers_of_tomorrow()

In [ ]:
# Test of lottery tickets (dev)
lotto_1 = Tickets(euromillions)

In [ ]:
# Test of Euromillions Analysis
start_time = time.time()

success = []
failure = []
euromillions = Criteria()
size = len(euromillions.scrap)

for draw in draw_generator(size):
    db_slice = euromillions.scrap.head(draw)
    euromillions.db = db_slice
    euromillions.groups_info()
    euromillions.apply_transformation()
    euromillions.count_skips()
    euromillions.skips_for_last_12_draws()
    euromillions.get_natural_rotations()
    euromillions.numbers_clasification()

    euromillions.year_criterion()
    euromillions.rotation_criterion()
    euromillions.position_criterion()
    euromillions.group_criterion()
    euromillions.numbers_of_tomorrow()

    row = euromillions.scrap.loc[draw,['nro1','nro2','nro3','nro4','nro5']]
    column = euromillions.recommended_numbers.loc[:,'numbers']
    if len(column) < 25:
        missing_rows = 25 - len(column)
        first_not_recommended_numbers = euromillions.not_recommended_numbers.head(missing_rows)
        column = pd.concat([euromillions.recommended_numbers['numbers'], first_not_recommended_numbers.iloc[:, 0]]).reset_index(drop=True)
    else:
        result = np.isin(column,row).sum()
        success.append(result)
        failure.append(5 - result)

    sys.stdout.write(f"\ri = {draw}")

c = Counter(success)
x = Counter(failure)

# Print quantity of hits per draw, with recommended numbers, and random numbers aside
for i in range(6):
    hits = (success.count(i)/len(success))*100
    print(f"{i} hits: {c[i]}\n{round(hits,2)}%")

random_succes = []
random_failure = []
for draw in draw_generator(size):
    db_resultados = db_slice.head(draw)
    row = euromillions.scrap.loc[draw,['nro1','nro2','nro3','nro4','nro5']]
    random_numbers = np.random.choice(range(1,51),size=25,replace=False)
    result = np.isin(row, random_numbers).sum()
    random_succes.append(result)
    random_failure.append(5-result)
    sys.stdout.write(f"\ri = {draw}")
    sys.stdout.flush()

y = Counter(random_succes)
z = Counter(random_failure)

# Print quantity of hits per draw, with recommended numbers, and random numbers aside
for e in range(6):
    nohits = (random_succes.count(e)/len(random_succes))*100
    print(f"{e} random hits: {y[e]}\n{round(nohits,2)}%")
